In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
player_playoff_stats = pd.read_excel("data_files/Playoff Player Stats.xlsx")
player_playoff_stats.head()

,Year,Age,Team,Lg,Round,W/L,Opp,G,W,L,...,PTS,Unnamed: 25,FG%,3P%,FT%,Unnamed: 29,MP.1,PTS.1,TRB.1,AST.1
0,1985.0,21.0,CHI,1985.0,EC1,L (1-3),MIL,4.0,1.0,3.0,...,117.0,NaN,0.436,0.125,0.828,NaN,42.8,29.3,5.8,8.5
1,1986.0,22.0,CHI,1986.0,EC1,L (0-3),BOS,3.0,0.0,3.0,...,131.0,NaN,0.505,1.000,0.872,NaN,45.0,43.7,6.3,5.7
2,1987.0,23.0,CHI,1987.0,EC1,L (0-3),BOS,3.0,0.0,3.0,...,107.0,NaN,0.417,0.400,0.897,NaN,42.7,35.7,7.0,6.0
3,1988.0,24.0,CHI,1988.0,EC1,W (3-2),CLE,5.0,3.0,2.0,...,226.0,NaN,0.559,NaN,0.918,NaN,43.4,45.2,5.4,4.8
4,1988.0,24.0,CHI,1988.0,ECS,L (1-4),DET,5.0,1.0,4.0,...,137.0,NaN,0.491,0.333,0.789,NaN,42.0,27.4,8.8,4.6
